# Проектная работа -  Создание и настройка дашборда

Автор: Чурюлин Евгений

## Описание проекта

Вы работаете аналитиком в Яндекс.Дзене. Почти всё ваше время занимает анализ пользовательского взаимодействия с карточками статей. Каждую карточку определяют её тема и источник (у него тоже есть тема).

***Примеры тем:*** 
* «Красота и здоровье»
* «Россия»
* «Путешествия»

Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+». Есть три способа взаимодействия пользователей с системой:
* Карточка отображена для пользователя (`show`)
* Пользователь кликнул на карточку (`click`)
* Пользователь просмотрел статью карточки (`view`)

Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы:
1. Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
2. Как много карточек генерируют источники с разными темами?
3. Как соотносятся темы карточек и темы источников?

На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.

## Задачи проекта

1. **Сделать дашборд** с использованием программы *Tablo*;

Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан для вас дата-инженерами. **Файл выгруженных данных для дашборда назовите `dash_visits.csv`**;

2. **Опубликовать дашборд** на сайте Tableau Public и подготовьте ссылку;

3. **Подготовить** файл с **презентацией** и ответить на вопросы

4. **Упоковать** все файлы в **zip-архив** и отправить на проверку ревьюеру.

*Дополнительно:* 

* В архиве могут быть добавлены дополнительные файлы, необходимые для корректной работы. 

* Размер до архивации не должен превышать 9 мб.

## Техническое задание проекта

* **Бизнес-задача:** анализ взаимодействия пользователей с карточками Яндекс.Дзен;

* **Насколько часто** предполагается **пользоваться дашбордом**: не реже, чем раз в неделю;


* Кто будет **основным пользователем** дашборда: менеджеры по анализу контента;


* **Состав данных** для дашборда:
    * История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    * Разбивка событий по темам источников;
    * Таблица соответствия тем источников темам карточек;

* По каким параметрам **данные должны группироваться**:
    * Дата и время;
    * Тема карточки;
    * Тема источника;
    * Возрастная группа;



* **Характер данных**:
    * История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    * Разбивка событий по темам источников — относительные величины (% событий);
    * Соответствия тем источников темам карточек - абсолютные величины;

* **Важность**: все графики имеют равную важность;

* **Источники данных** для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица `log_raw`);

* **База данных**, в которой будут **храниться агрегированные данные**: дополнительные агрегированные таблицы в БД `zen`;

* **Частота обновления данных**: один раз в сутки, в `полночь по UTC`;

* **Какие графики** должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):


![dash](https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-06T160925.436_1594041010.png)


## Основной проект

### Загрузка данных

Для создания дашборда выполните вот эти шаги:
1. Скачайте файл https://storage.yandexcloud.net/cloud-certs/CA.pem в папку с файлом .ipynb, в котором вы будете работать с базой
2. Создайте коннекцию к базе, скопировав код ниже:

In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# Задаём параметры подключения к БД,
db_config = {'user': 'praktikum_student'                        , # имя пользователя
             'pwd' : 'Sdf4$2;d-d30pp'                           , # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432                                       , # порт подключения
             'db'  : 'data-analyst-zen-project-db'              } # название базы данных

# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'] ,
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']  )
# Подключаемся к БД.
engine = create_engine(connection_string)


# Формируем sql-запрос.
query = ''' SELECT *
            FROM dash_visits
        '''

# Выполняем запрос и сохраняем результат выполнения в DataFrame.
data_raw = pd.io.sql.read_sql(query, con = engine) 

   record_id item_topic source_topic age_segment                  dt  visits
0    1040597     Деньги         Авто       18-25 2019-09-24 18:32:00       3
1    1040598     Деньги         Авто       18-25 2019-09-24 18:35:00       1
2    1040599     Деньги         Авто       18-25 2019-09-24 18:54:00       4
3    1040600     Деньги         Авто       18-25 2019-09-24 18:55:00      17
4    1040601     Деньги         Авто       18-25 2019-09-24 18:56:00      27


In [7]:
# Get common information about datasets
def get_info(df, df_name):
    '''
    df : Dataframe,
        The dataset with data for the project
    df_name : Objects
        The name of the dataset for information
        
    Return:
        There are no return fields
    '''
    print(f'Common information about - {df_name}')
    display(df.head())
    df.info()
    print(df.columns, '\n')   
    print(f'Numbers of NaN values in the dataset - {df_name}', '\n')
    print(df.isnull().sum())
    print(f'Numbers of duplicates (explicit)in the dataset - {df_name}', '\n')
    print(df.duplicated().sum())
     

# Просмотр общей информации о файле
print(get_info(data_raw, 'data_raw')) 

Common information about - data_raw


,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB
Index(['record_id', 'item_topic', 'source_topic', 'age_segment', 'dt',
       'visits'],
      dtype='object') 

Numbers of NaN values in the dataset - data_raw 

record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64
Numbers of duplicates (explicit)in the dataset - data_raw 

0
None


**Вывод:** В загруженом файле нет дубликатов, пропусков. Всего 30745 строк. Приступаем к следующему этапу работы

In [8]:
# Выгружаем данные в csv
data_raw.to_csv('C:/Users/Churiulin/projects/' + 'dash_visits.csv', sep=',')

Для запуска данного скрипта по расписанию на моем локальном компьютере можно использовать следующую команду:

In [ ]:
0 6 * * 1 python -u -W ignore /mnt/hgst/scared2/script_project.py --start_dt=$(date +\%Y-\%m-\%d\ 00:00:00 -d "1 week ago") >> /mnt/hgst/scared2/logs/script_project_$(date +\%Y-\%m-\%d).log 2>&1

## Tablo

Ссылка на дашборд: https://public.tableau.com/app/profile/evgenii4045/viz/Project_Tablo/Dashboard1

Ссылка на презентацию: https://drive.google.com/file/d/1Xgoqs6e06QrPZ2jNLEgMZjfavOGx16YQ/view?usp=sharing